In [1]:
import numpy as np
import copy

import heapq

from collections import defaultdict

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt


In [2]:
with open('input_day_17.txt', 'r') as file:
    input_string = file.read()


test_input_string = """
Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0
""".strip()


In [3]:
register_str, program_str = input_string.split("\n\n")
register_rows = register_str.split("\n")
register_vals = [int(row.split(":")[1]) for row in register_rows]

registers = {"A": register_vals[0], "B": register_vals[1], "C": register_vals[2]}

program = [int(x) for x in program_str.split(":")[1].strip().split(",")]

instruction_pointer = 0

def combo(x, registers):
    if x == 4:
        return registers["A"]
    if x == 5:
        return registers["B"]
    if x == 6:
        return registers["C"]
    return x

def test(A, debug=False):

    curr_registers = copy.deepcopy(registers)
    curr_registers["A"] = A

    output = []
    instruction_pointer = 0

    while True:

        if instruction_pointer + 1 > len(program):
            #print("\nhalt")
            break

        opcode, operand = program[instruction_pointer], program[instruction_pointer + 1]

        if debug:
            print("\npointer:", instruction_pointer)
            print("registers:", curr_registers["A"], curr_registers["B"], curr_registers["C"])
            print("operation, operand:", [
                "adv (A = A // (2**combo(operand)))", 
                "bxl (B = B ^ operand)", 
                "bst (B = combo(operand) % 8)", 
                "jnz (if A: instruction_pointer = operand)", 
                "bxc (B = B ^ C)", 
                "out (output combo(operand) % 8))", 
                "bdv (B = A // (2**combo(operand)))", 
                "cdv (C = A // (2**combo(operand)))"
                ][opcode], operand)
            print("output:", output)

        if opcode == 0: # adv
            curr_registers["A"] = curr_registers["A"] // (2**combo(operand, curr_registers))
            instruction_pointer += 2
        elif opcode == 1: # bxl
            curr_registers["B"] = curr_registers["B"] ^ operand
            instruction_pointer += 2
        elif opcode == 2: # bst
            curr_registers["B"] = combo(operand, curr_registers) % 8
            instruction_pointer += 2
        elif opcode == 3: # jnz
            if curr_registers["A"] == 0:
                instruction_pointer += 2
            else:
                instruction_pointer = operand
        elif opcode == 4: # bxc
            curr_registers["B"] = curr_registers["B"] ^ curr_registers["C"]
            instruction_pointer += 2
        elif opcode == 5: # out
            output.append(combo(operand, curr_registers) % 8)
            instruction_pointer += 2
        elif opcode == 6: # bdv
            curr_registers["B"] = curr_registers["A"] // (2**combo(operand, curr_registers))
            instruction_pointer += 2
        elif opcode == 7: # cdv
            curr_registers["C"] = curr_registers["A"] // (2**combo(operand, curr_registers))
            instruction_pointer += 2

    return output

output = test(registers["A"], debug=True)

s = ""
for x in output:
    s += str(x)
    s += ","
s = s[:-1]
print(s)




pointer: 0
registers: 27575648 0 0
operation, operand: bst (B = combo(operand) % 8) 4
output: []

pointer: 2
registers: 27575648 0 0
operation, operand: bxl (B = B ^ operand) 2
output: []

pointer: 4
registers: 27575648 2 0
operation, operand: cdv (C = A // (2**combo(operand))) 5
output: []

pointer: 6
registers: 27575648 2 6893912
operation, operand: bxc (B = B ^ C) 1
output: []

pointer: 8
registers: 27575648 6893914 6893912
operation, operand: bxl (B = B ^ operand) 3
output: []

pointer: 10
registers: 27575648 6893913 6893912
operation, operand: out (output combo(operand) % 8)) 5
output: []

pointer: 12
registers: 27575648 6893913 6893912
operation, operand: adv (A = A // (2**combo(operand))) 3
output: [1]

pointer: 14
registers: 3446956 6893913 6893912
operation, operand: jnz (if A: instruction_pointer = operand) 0
output: [1]

pointer: 0
registers: 3446956 6893913 6893912
operation, operand: bst (B = combo(operand) % 8) 4
output: [1]

pointer: 2
registers: 3446956 4 6893912
opera

In [4]:
# at pointer 0:
#           B = A % 8

# at pointer 2:
#           B = B ^ 2
#           B = (A % 8) ^ 2
#           B = 

# at pointer 4:
#           C = A // (2**B)
#           C = A // (2**((A % 8) ^ 2))

# at pointer 6:
#           B = B ^ C = ((A % 8) ^ 2) ^ (A // (2**((A % 8) ^ 2)))

# at pointer 8:
#           B = B ^ 3 = (((A % 8) ^ 2) ^ (A // (2**((A % 8) ^ 2)))) ^ 3

# at pointer 10: 
#           outputs B % 8, has to get 2 -> ((((A % 8) ^ 2) ^ (A // (2**((A % 8) ^ 2)))) ^ 3) % 8 == 2

# at pointer 12:
#           A = A // 8

# at pointer 14:
#           A != 0 (has to jump, otherwise fails)
#           goes to instruction 0, starts over again


# important part: ((((A % 8) ^ 2) ^ (A // (2**((A % 8) ^ 2)))) ^ 3) % 8

# so, each output should mostly depend on A % 8
# The A floordiv part matters as well, but only the last three digits of the result of that division
# so, the last few digits should matter each time, and then we shift over


In [15]:
# working backwards, so the floordiv part from the bigger number is already considered

successes = []

def add_to_A_in_order_to_print_program(A):

    for a in range(8):
        output = test(A + a)

        if output == program:
            print("success!", A + a)
            successes.append(A+a)

        if output == program[-len(output):]:
            add_to_A_in_order_to_print_program((A + a)*8)

add_to_A_in_order_to_print_program(0)

print(min(successes))

real success! 37221261688308
real success! 37221265882612
real success! 37221266013684
real success! 38878582193652
real success! 38878586387956
real success! 38878586519028
real success! 38878590582260
real success! 38886098386420
real success! 38886102580724
real success! 38886102711796
real success! 38886106775028
37221261688308
